<a href="https://colab.research.google.com/github/rreimche/genai-exam/blob/main/LitRev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites:

- Google Colab
- Access to a Google Colab Secret named "HF_API_TOKEN" containing a Huggingface API token with read access.

In [1]:
#@title Settings
from google.colab import userdata

download_dir = "papers"  # directory name to download papers to revie to
huggingface_apikey = userdata.get('HF_API_TOKEN')  # use colab secrets to store the huggingface api key
groq_key = userdata.get('GROQ_KEY')  #  use colab secrets to store the huggingface api key


In [ ]:
#@title Declare bibtex references and parse them into a variable for later usage


bibtext_references = """


@misc{qian2024chatdevcommunicativeagentssoftware,
      title={ChatDev: Communicative Agents for Software Development},
      author={Chen Qian and Wei Liu and Hongzhang Liu and Nuo Chen and Yufan Dang and Jiahao Li and Cheng Yang and Weize Chen and Yusheng Su and Xin Cong and Juyuan Xu and Dahai Li and Zhiyuan Liu and Maosong Sun},
      year={2024},
      eprint={2307.07924},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2307.07924},
},

@misc{nguyen2024agilecoderdynamiccollaborativeagents,
      title={AgileCoder: Dynamic Collaborative Agents for Software Development based on Agile Methodology},
      author={Minh Huynh Nguyen and Thang Phan Chau and Phong X. Nguyen and Nghi D. Q. Bui},
      year={2024},
      eprint={2406.11912},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2406.11912},
},


"""



In [2]:
#@title Install dependencies
!pip install -q autogen arxiv scholarly crossrefapi beautifulsoup4 requests cloudscraper pymupdf nltk autogen-agentchat autogen-ext[openai] groq pymupdf;
!pip install -q --pre bibtexparser;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.5/642.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.3 MB/s eta 0:00:00
   ━━━

In [25]:
#@title Preparation for agents: model client connections
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core.tools import FunctionTool
from autogen_core.models import UserMessage
#from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# INSTANTIATE MODEL CLIENTS

# This client will be used for paper summarization
text_model_client = OpenAIChatCompletionClient(
    #model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    model="mistralai/Mistral-7B-Instruct-v0.3",
    base_url="https://router.huggingface.co/hf-inference/v1",
    api_key=huggingface_apikey,
    model_info={
        "vision": False,
        "function_calling": False,
        "json_output": False,
        "family": "unknown",
    },
)

# We need another inference point for downloader agent,
# because huggingface can't serve reflection on tool use
# so that autogen agent understands it
tool_model_client = OpenAIChatCompletionClient(
    model="llama3-70b-8192",
    #model="deepseek-r1-distill-qwen-32b",
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)


# CHECK CONNECTION

async def check_model_connect() -> None:
    messages = [
        UserMessage(content="What is the capital of France?", source="user"),
    ]
    response = await text_model_client.create(messages=messages)
    response_downloader = await tool_model_client.create(messages=messages)

    print(response.content)
    print(response_downloader.content)

await check_model_connect()

The capital of France is Paris. It is one of the most famous cities in the world, known for its historic sites such as the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Arc de Triomphe. Paris is also renowned for its art, fashion, gastronomy, and culture.
The capital of France is Paris.


In [ ]:
#@title Downloader agent
import arxiv
import os
import datetime
import json
import sys
from dataclasses import dataclass
from autogen_core.models import ChatCompletionClient
from autogen_core import message_handler

# HELPER AND TOOL FUNCTIONS

def maybe_create_dir(donwload_dir: str) -> None:
    """
        A helper function to create the downloads directory
        in colab root if the directory is not yet present.
    """

    try:
        os.makedirs(donwload_dir, exist_ok=True)
        #print(f"Directory '{download_dir}' created successfully.")
    except OSError as e:
        raise Exception(f"Error creating directory '{download_dir}': {e}")

def download_paper(eprint_id: str) -> str:
    """
        Receives a eprint_id of a paper from arxiv, downloads the specified paper
        and saves it in downloads directory.
        The filepath of the downloaded paper is in the resulting dictionary
        with the key "filepath".

        :param eprint_id: string, the arxiv eprint_id.
        :return: string with filepath of the downloaded paper.
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"

    result = {
        "eprint_id": eprint_id,
        "success": False,
        "filepath": None,
        }


    if eprint_id:
        #print(f"{datetime.datetime.now()} - Trying arXiv for: {eprint_id}")
        try:
            client = arxiv.Client()
            search = arxiv.Search(id_list=[eprint_id], max_results=1)

            #arxiv_result = next(search.results(), None)
            arxiv_result = next(client.results(search))
            if arxiv_result:
                #print(f"  Found on arXiv: {arxiv_result.title}")
                maybe_create_dir(download_dir)
                filepath = os.path.join(download_dir, f"{arxiv_result.title.replace(' ', '_')}.pdf")
                arxiv_result.download_pdf(filename=filepath)
                result["success"] = True
                result["filepath"] = filepath
                #print(f"  Download successful: {filepath}")
            else:
                # print(f"  No arXiv result found for ID {eprint_id}.")
                raise Exception(f"ERROR: No arXiv result found for ID {eprint_id}.")
                #return f"ERROR: No arXiv result found for ID {eprint_id}."

        except Exception as e:
            #print(f"arXiv download failed for: {eprint_id}: {e}")
            raise Exception(f"ERROR: arXiv download failed for: {eprint_id}: {e}")
            #return f"ERROR: arXiv download failed for: {eprint_id}: {e}"

    # Check if the download was NOT successful
    if not result["success"]:
        #print(f"{datetime.datetime.now()} - Download failed or not attempted for: {eprint_id}")
        raise Exception(f"ERROR: Download failed or not attempted for: {eprint_id}")
        #return f"ERROR: Download failed or not attempted for: {eprint_id}"

    return result["filepath"]


# DOWNLOADER AGENT WITH TOOL

def create_downloader(given_name: str) -> AssistantAgent:
    downloader_tool = FunctionTool(download_paper, description="Given a string denoting an eprint_id of a page on arxiv, downloads a paper")

    return AssistantAgent(
        name=given_name,
        description="An agent that uses downloader tool to download a paper from Arxiv identified by a eprint_id",
        model_client=tool_model_client,
        tools=[downloader_tool],
        reflect_on_tool_use=True,
        system_message="""
            Use downloader tool to download specified paper(s) from arxiv.
            Here is an example of how you must respond in success case:
              'mypapers/veryinterstingpaper.pdf'

            Very important: You MUST return the full filepath of the downloaded
            paper EXACTLY as it was given by the downloader tool.

            If download was not successful, respond like this:
              'ERROR: Download for paper with eprint id XXX failed for the following reason: REASON'.
          """,
    )

# CHECK IF DOWNLOAD WORKS

async def downloader_run() -> None:
    response = await create_downloader("Test").on_messages(
        [TextMessage(content="Download the paper with eprint_id 2402.01411", source="user")],
        cancellation_token=CancellationToken(),
    )


    #print(response.inner_messages)
    #print(response.chat_message)
    print(response.chat_message.content)



# Use asyncio.run(assistant_run()) when running in a script.
#await downloader_run()


In [33]:
#@title Agents to get paper text: Informant and Parser
import arxiv
from typing import Dict
import pymupdf
import requests
from autogen_agentchat.ui import Console

def get_arxiv_data(eprint_id: str) -> Dict[str, str]:
    """
    Given arxiv eprint_id of a scientific paper, returns title of the paper
    along with the PDF URL.

    :param eprint_id: string, the arxiv eprint_id.
    :return: dictionary with keys "title" and "pdf_url".
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"

    client = arxiv.Client()
    search = arxiv.Search(id_list=[eprint_id], max_results=1)

    try:

        arxiv_result = next(client.results(search))
        if arxiv_result:
            return {
                "title": arxiv_result.title,
                "pdf_url": arxiv_result.pdf_url,
            }
        else:
            raise Exception(f"ERROR: No arXiv result found for ID {eprint_id}.")

    except Exception as e:
        raise Exception(f"ERROR: arXiv download failed for: {eprint_id}: {e}")

tool_paperinfo = FunctionTool(
    get_arxiv_data,
    description="Given a string denoting an eprint_id of a page on arxiv, returns title of the paper along with the PDF URL"
)

def parse_paper(url: str) -> str:
    """
    Given URL of a PDF as a string, returns the contents as a string

    :param url: string, the URL of the PDF.
    :return: string with the contents of the PDF.
    """

    try:

        r = requests.get(url)
        data = r.content
        doc = pymupdf.Document(stream=data)
        text = chr(12).join([page.get_text() for page in doc])
        return text

    except Exception as e:
        # print(f"Error parsing {title} ({filepath}): {e}")
        raise Exception(f"ERROR: Error downloading or parsing the paper at {url}: {e}")

tool_parsepdf = FunctionTool(
    parse_paper,
    description = "Given URL of a PDF as a string, returns the contents as a string"
)

paper_informant = AssistantAgent(
    name="informant",
    description="Given an arxiv eprint_id, gets the paper metadata, including title and PDF URL",
    model_client=tool_model_client,
    tools=[tool_paperinfo],
    reflect_on_tool_use=True,
    system_message="""
      You are a helpful assistant. You are given a string
      that denotes an eprint_id of a paper published on arxiv.

      You must use the provided tools to get paper information from Arxiv.
    """
)

paper_parser = AssistantAgent(
    name="parser",
    description="Given a URL of a paper PDF file, gets the file, parses it and returns the text",
    model_client=tool_model_client,
    tools=[tool_parsepdf],
    reflect_on_tool_use=False,
    system_message="""
      You are a helpful assistant. You are given a string
      that denotes a URL of a paper PDF file.

      Use the provided tool to parse the PDF file and answer with the text.
    """
)



# CHECK IF WORKS

async def get_paper_text_run() -> None:
    response_i = await paper_informant.on_messages(
        [TextMessage(content="Give me the infos of the paper with eprint_id 2307.07924", source="user")],
        cancellation_token=CancellationToken(),
    )

    prev_answer = response_i.chat_message.content

    #print(prev_answer)

    response_p = await paper_parser.on_messages(
        [TextMessage(content=f"Give me the text of paper, which url is specified in the following text: {prev_answer}", source="user")],
        cancellation_token=CancellationToken(),
    )

    # for deeper debugging
    #await Console(
    #    paper_parser.on_messages_stream(
    #        [TextMessage(content=f"Give me the text of the related paper: {prev_answer}", source="user")],
    #        cancellation_token=CancellationToken(),
    #    ),
    #    output_stats=True,  # Enable stats printing.
    #)


    #print(response.inner_messages)
    #print(response.chat_message)
    #print(response.chat_message.content)

    return response_p.chat_message.content



# Use asyncio.run(assistant_run()) when running in a script.
test_paper_text = await get_paper_text_run()

print(test_paper_text)




ChatDev: Communicative Agents for Software Development
Chen Qian⋆
Wei Liu⋆
Hongzhang Liu♠
Nuo Chen⋆
Yufan Dang⋆
Jiahao Li⋆
Cheng Yang♣
Weize Chen⋆
Yusheng Su⋆
Xin Cong⋆
Juyuan Xu⋆
Dahai Li♦
Zhiyuan Liu⋆B
Maosong Sun⋆B
⋆Tsinghua University
♠The University of Sydney
♣BUPT
♦Modelbest Inc.
qianc62@gmail.com
liuzy@tsinghua.edu.cn
sms@tsinghua.edu.cn
Abstract
Software development is a complex task that
necessitates cooperation among multiple mem-
bers with diverse skills. Numerous studies used
deep learning to improve specific phases in a
waterfall model, such as design, coding, and
testing.
However, the deep learning model
in each phase requires unique designs, lead-
ing to technical inconsistencies across various
phases, which results in a fragmented and in-
effective development process. In this paper,
we introduce ChatDev, a chat-powered soft-
ware development framework in which special-
ized agents driven by large language models
(LLMs) are guided in what to communicate
(via chat chain)

In [43]:
#@title Conceptualizer

conceptualizer = AssistantAgent(
        name="conceptualizer",
        description="""
          Given a text of a scientific paper, this agent will return
          a conceptualization -- a list of topics discussed in the paper
          as well as the key contributions delivered.
        """,
        model_client=text_model_client,
        system_message="""
          You are a professor of computer science specialized in multi agent systems
          and generative AI. Given a text of a scientific paper, write a conceptualization
          of it -- provide the title as well as the structure of the paper and key contributions delivered.
          Write structured, using headers and bullet points. Provide short descriptions
          of key contributions. Fit your answer in 500 words.
        """
    )


# CHECK IF CONCEPTUALIZATION WORKS

async def concept_run(paper_text: str) -> None:
    response = await conceptualizer.on_messages(
        [TextMessage(content=paper_text, source="user")],
        cancellation_token=CancellationToken(),
    )

    #print(paper_text)
    #print(response.inner_messages)
    print(response.chat_message.models_usage)
    #print(response.chat_message.content)
    return response.chat_message.content



# Use asyncio.run(assistant_run()) when running in a script.
conceptualization_test = await concept_run(test_paper_text)  # we'll need the variable later for testing of other agents
print(conceptualization_test)

RequestUsage(prompt_tokens=17131, completion_tokens=664)
The paper introduces ChatDev, a framework for software development using a chat-based system powered by language models (LLMs). The system is composed of multiple specialized agents, each with a specific role (e.g., programmer or reviewer) that work collaboratively through multi-turn dialogues, using unique roles, communication patterns, and context awareness to improve software's quality in the design, coding, and testing phases. The paper highlights that the use of linguistic communication facilitates multi-agent collaboration and provides the novel concept of "Communicative Dehallucination" to address coding hallucinations. Results indicate that the proposed paradigm enhances the completeness, executability, and consistency of software generated, and outperforms existing LLM-powered software development methods. The findings suggest that using cooperative agents through effective communication for software optimization is a pr

In [44]:
#@title Summarizer agent


summarizer_agent = AssistantAgent(
    name="summarizer",
    description="""
      Expects a conceptualization of a paper (title, structure of the paper, key points and contributions)
      and write a summary of the paper.
    """,
    model_client=text_model_client,
    system_message="""
        You are a professor of computer science
        and also qualified and experienced reviewer and critic of scientific papers.
        You will be given a conceptualization of a paper (title, structure of the paper, key points and contributions).
        Summarize the paper using the points provided in conceptualisation.
        You must fit your summarization in 500 words.
        It is very important to depict the key contributions of the paper.
"""
)

# CHECK IF SUMMARIZING WORKS

async def summarizer_run(paper_text: str, conceptualization: str) -> None:

    prompt = f"""
      ------start conceptualization-------
      {conceptualization}
      ------end conceptualization-------
      """


    response = await summarizer_agent.on_messages(
        [TextMessage(content=prompt, source="user")],
        cancellation_token=CancellationToken(),
    )

    #print(paper_text)
    #print(response.inner_messages)
    print(response.chat_message.models_usage)
    return response.chat_message.content


summarization = await summarizer_run(test_paper_text, conceptualization_test)
print(summarization)

RequestUsage(prompt_tokens=787, completion_tokens=449)
Title: "ChatDev: A Multi-Agent Framework for Software Development Using conversational AI"

This paper presents the development and introduction of ChatDev, a novel, multi-agent software development framework that leverages the power of Language Models (LLMs) to streamline the software development process. The framework revolves around a chat-based system, where multiple agents with specialized roles collaborate in a conversational manner to produce high-quality software.

The system's architecture includes agents taking on roles such as the CEO, CTO, programmer, reviewer, and tester, thereby enhancing their contributions to the various stages of software development. The use of an organized chat-chain system overlooks the fluid communication, optimizing the overall development process through a unified language-based exchange of ideas.

Introducing the concept of "Communicative Dehallucination," this paper addresses the coding hal